In [ ]:
#%cd /content/drive/MyDrive/recvis21_a3-master
!pip install -r requirements.txt

In [ ]:
import argparse
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets
from torch.autograd import Variable
from tqdm import tqdm
import zipfile
import os

from google.colab import drive
drive.mount('/content/drive/', force_remount=True)
os.chdir('/content/drive/My Drive/')

import torchvision.transforms as transforms

# once the images are loaded, how do we pre-process them before being passed into the network
# by default, we resize the images to 64 x 64 in size
# and normalize them to mean = 0 and standard-deviation = 1 based on statistics collected from
# the training set
data_transforms_train = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomRotation(degrees=45),
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
])

data_transforms_val = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
        ])


Mounted at /content/drive/


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models

nclasses = 20 

def freeze(model, block=3):
    modules = list(model.children())[:-1]
    for module in modules[:-block]:
        for p in module.parameters() : p.requires_grad = False
    return nn.Sequential(*modules)
           

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.decision1, self.decision2 = nn.Linear(2048, 512), nn.Linear(2048, 512)
        self.final = nn.Linear(1024, nclasses)
        self.m1, self.m2 = (freeze(models.resnet152(pretrained=True), block=0),
                            freeze(models.resnet101(pretrained=True), block=0))

    def forward(self, x):
        f1, f2 = self.m1(x).view(-1,2048), self.m2(x).view(-1,2048)
        f1, f2 = self.decision1(f1), self.decision2(f2)
        features = torch.cat([f1, f2], dim=1)
        output = self.final(features)
        
        return output
    
    
    

    

Downloading: "https://download.pytorch.org/models/resnet152-394f9c45.pth" to /root/.cache/torch/hub/checkpoints/resnet152-394f9c45.pth


  0%|          | 0.00/230M [00:00<?, ?B/s]

Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth


  0%|          | 0.00/171M [00:00<?, ?B/s]

In [ ]:
import argparse
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets
from torch.autograd import Variable
from tqdm import tqdm
import numpy as np

# Training settings
parser = argparse.ArgumentParser(description='RecVis A3 training script')
parser.add_argument('--data', type=str, default='bird_dataset', metavar='D',
                    help="folder where data is located. train_images/ and val_images/ need to be found in the folder")
parser.add_argument('--batch-size', type=int, default=4, metavar='B',
                    help='input batch size for training (default: 64)')
parser.add_argument('--epochs', type=int, default=37, metavar='N',
                    help='number of epochs to train (default: 10)')
parser.add_argument('--lr', type=float, default=0.0005, metavar='LR',
                    help='learning rate (default: 0.0005)')
parser.add_argument('--momentum', type=float, default=0.8, metavar='M',
                    help='SGD momentum (default: 0.5)')
parser.add_argument('--seed', type=int, default=1, metavar='S',
                    help='random seed (default: 1)')
parser.add_argument('--log-interval', type=int, default=10, metavar='N',
                    help='how many batches to wait before logging training status')
parser.add_argument('--experiment', type=str, default='experiment', metavar='E',
                    help='folder where experiment outputs are located.')
parser.add_argument('-f')
args = parser.parse_args()
use_cuda = torch.cuda.is_available()
torch.manual_seed(args.seed)

# Create experiment folder
if not os.path.isdir(args.experiment):
    os.makedirs(args.experiment)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

################ Data initialization and loading ################

train_loader = torch.utils.data.DataLoader(
    datasets.ImageFolder(args.data + '/train_images',
                         transform=data_transforms_train),
    batch_size=args.batch_size, shuffle=True, num_workers=1)
val_loader = torch.utils.data.DataLoader(
    datasets.ImageFolder(args.data + '/val_images',
                         transform=data_transforms_val),
    batch_size=args.batch_size, shuffle=False, num_workers=1)

test_loader = torch.utils.data.DataLoader(
    datasets.ImageFolder(args.data + '/test_images',transform=data_transforms_val),
    batch_size=1, shuffle=False, num_workers=1)


######################   Model creation   ######################

# Neural network and optimizer
# We define neural net in model.py so that it can be reused by the evaluate.py script
model = Net()
if use_cuda:
    print('Using GPU')
    model.cuda()
else:
    print('Using CPU')

optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=args.momentum)

def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        optimizer.zero_grad()
        output = model(data)
        criterion = torch.nn.CrossEntropyLoss(reduction='mean')
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data.item()))

def validation():
    model.eval()
    validation_loss = 0
    correct = 0
    for data, target in val_loader:
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        output = model(data)
        # sum up batch loss
        criterion = torch.nn.CrossEntropyLoss(reduction='mean')
        validation_loss += criterion(output, target).data.item()
        # get the index of the max log-probability
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    validation_loss /= len(val_loader.dataset)
    print('\nValidation set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
        validation_loss, correct, len(val_loader.dataset),
        100. * correct / len(val_loader.dataset)))


for epoch in range(1, args.epochs + 1):
    train(epoch)
    validation()
    model_file = args.experiment + '/model_' + str(epoch) + '.pth'
    torch.save(model.state_dict(), model_file)
    print('Saved model to ' + model_file + '. You can run `python evaluate.py --model ' + model_file + '` to generate the Kaggle formatted csv file\n')


Using GPU
Train Epoch: 1 [0/1082 (0%)]	Loss: 3.005311
Train Epoch: 1 [40/1082 (4%)]	Loss: 3.098603
Train Epoch: 1 [80/1082 (7%)]	Loss: 2.995406
Train Epoch: 1 [120/1082 (11%)]	Loss: 2.885162
Train Epoch: 1 [160/1082 (15%)]	Loss: 3.106729
Train Epoch: 1 [200/1082 (18%)]	Loss: 3.198532
Train Epoch: 1 [240/1082 (22%)]	Loss: 2.986267
Train Epoch: 1 [280/1082 (26%)]	Loss: 3.064450
Train Epoch: 1 [320/1082 (30%)]	Loss: 2.998599
Train Epoch: 1 [360/1082 (33%)]	Loss: 3.231786
Train Epoch: 1 [400/1082 (37%)]	Loss: 2.887415
Train Epoch: 1 [440/1082 (41%)]	Loss: 3.013239
Train Epoch: 1 [480/1082 (44%)]	Loss: 3.020957
Train Epoch: 1 [520/1082 (48%)]	Loss: 3.039126
Train Epoch: 1 [560/1082 (52%)]	Loss: 2.970952
Train Epoch: 1 [600/1082 (55%)]	Loss: 3.165469
Train Epoch: 1 [640/1082 (59%)]	Loss: 2.931123
Train Epoch: 1 [680/1082 (63%)]	Loss: 2.935833
Train Epoch: 1 [720/1082 (66%)]	Loss: 2.993508
Train Epoch: 1 [760/1082 (70%)]	Loss: 2.872622
Train Epoch: 1 [800/1082 (74%)]	Loss: 2.838984
Train Epoc

# Test

In [ ]:
#!python /content/evaluate.py --model experiment/model_24.pth
#python /content/evaluate.py --model experiment/model_36.pth
!python /content/evaluate.py --model experiment/model_24.pth

Traceback (most recent call last):
  File "/content/evaluate.py", line 23, in <module>
    model.load_state_dict(state_dict)
  File "/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py", line 1483, in load_state_dict
    self.__class__.__name__, "\n\t".join(error_msgs)))
RuntimeError: Error(s) in loading state_dict for Net:
	Missing key(s) in state_dict: "conv1.weight", "conv1.bias", "conv2.weight", "conv2.bias", "conv3.weight", "conv3.bias", "fc1.weight", "fc1.bias", "fc2.weight", "fc2.bias". 
	Unexpected key(s) in state_dict: "decision1.weight", "decision1.bias", "decision2.weight", "decision2.bias", "final.weight", "final.bias", "m1.0.weight", "m1.1.weight", "m1.1.bias", "m1.1.running_mean", "m1.1.running_var", "m1.1.num_batches_tracked", "m1.4.0.conv1.weight", "m1.4.0.bn1.weight", "m1.4.0.bn1.bias", "m1.4.0.bn1.running_mean", "m1.4.0.bn1.running_var", "m1.4.0.bn1.num_batches_tracked", "m1.4.0.conv2.weight", "m1.4.0.bn2.weight", "m1.4.0.bn2.bias", "m1.4.0.bn2.running_m

In [ ]:
# Remember : model = Net()
preds = np.array([])
model.eval()
with torch.no_grad():
    for i, (data, labels) in tqdm(enumerate(test_loader, 0)):
        data, labels = data.to(device), labels.to(device)
        output1 = model(data)
        sm = nn.Softmax(dim=1)(output1)
        pred = sm.max(1, keepdim=True)[1]    
        preds = np.hstack((preds, torch.squeeze(pred).cpu().numpy()))

517it [01:25,  6.03it/s]


In [ ]:
f = open("92%_14h23_submission_du_22_11_21.csv", "w")
f.write("Id,Category\n")
for (n,_),p in zip(test_loader.dataset.samples,preds):
    f.write("{},{}\n".format(n.split('/')[-1].split('.')[0], int(p)))
f.close()